In [1]:
import numpy as np
from scipy.optimize import fsolve, root
import warnings

In [2]:
def get_coupled_steady_state(
    # TF gene parameters
    k_on_TF,
    k_off_TF,
    basal_TF_rate,
    TF_mRNA_degradation_rate,
    splicing_rate_TF,
    TF_protein_production_rate,
    TF_protein_degradation_rate,
    
    # Target gene parameters
    k_on_Target,
    k_off_Target,
    basal_Target_rate,
    Target_mRNA_degradation_rate,
    splicing_rate_Target,
    Target_protein_production_rate,
    Target_protein_degradation_rate,
    max_effect=16.0,       # Maximum effect of regulation (can be adjusted)
    # Hill function parameters - can be fixed values or 'auto' for self-consistent
    K_TF='auto',          # Half-saturation for Target protein regulating TF
    K_Target='auto',      # Half-saturation for TF protein regulating Target
    n_TF=2,               # Hill coefficient for Target → TF regulation
    n_Target=2,           # Hill coefficient for TF → Target regulation
    
    # Solver options
    initial_guess=None,
    method='hybr',
    find_all_solutions=False,
    max_attempts=100
):
    """
    Find steady-state protein and mRNA levels for two mutually regulating genes.
    
    Parameters:
    -----------
    Gene-specific parameters for TF and Target genes including:
    - Transcriptional bursting (k_on, k_off)
    - Transcription rates (basal, max)
    - mRNA processing (degradation, splicing)
    - Protein production and degradation
    
    Hill function parameters:
    - K_TF, K_Target: half-saturation constants ('auto' sets K = steady-state protein level)
    - n_TF, n_Target: Hill coefficients (cooperativity)
    
    Returns:
    --------
    dict: Steady-state values for proteins and mRNAs, plus solver info
    """
    
    def steady_state_system(vars):
        """Define the coupled steady-state equations with self-consistent K values"""
        TF_protein, Target_protein = vars
        
        # Ensure non-negative protein levels
        TF_protein = max(TF_protein, 1e-12)
        Target_protein = max(Target_protein, 1e-12)
        
        # Set K values - either fixed or self-consistent
        K_TF_val = TF_protein if K_TF == 'auto' else K_TF
        K_Target_val = Target_protein if K_Target == 'auto' else K_Target
        
        # Hill functions for regulation
        # Target protein regulates TF gene (K_TF is the TF protein steady state)
        Target_hill = (Target_protein**n_TF) / (K_TF_val**n_TF + Target_protein**n_TF)
        
        # TF protein regulates Target gene (K_Target is the Target protein steady state)
        TF_hill = (TF_protein**n_Target) / (K_Target_val**n_Target + TF_protein**n_Target)
        
        # Effective k_on rates modulated by Hill functions
        k_on_TF_effective = k_on_TF #* max_effect * Target_hill
        k_on_Target_effective = k_on_Target * max_effect * TF_hill
        
        # TF gene expression pathway with effective k_on
        TF_bursting_fraction = k_on_TF_effective / (k_on_TF_effective + k_off_TF)
        TF_transcription_rate = basal_TF_rate
        TF_unspliced_mRNA = (TF_transcription_rate * TF_bursting_fraction / 
                            (TF_mRNA_degradation_rate + splicing_rate_TF))
        TF_spliced_mRNA = (TF_unspliced_mRNA * splicing_rate_TF / 
                          TF_mRNA_degradation_rate)
        TF_protein_predicted = (TF_spliced_mRNA * TF_protein_production_rate / 
                               TF_protein_degradation_rate)
        
        # Target gene expression pathway with effective k_on
        Target_bursting_fraction = k_on_Target_effective / (k_on_Target_effective + k_off_Target)
        Target_transcription_rate = basal_Target_rate
        Target_unspliced_mRNA = (Target_transcription_rate * Target_bursting_fraction / 
                                (Target_mRNA_degradation_rate + splicing_rate_Target))
        Target_spliced_mRNA = (Target_unspliced_mRNA * splicing_rate_Target / 
                              Target_mRNA_degradation_rate)
        Target_protein_predicted = (Target_spliced_mRNA * Target_protein_production_rate / 
                                   Target_protein_degradation_rate)
        
        # Return residuals (should be zero at steady state)
        residual_TF = TF_protein - TF_protein_predicted
        residual_Target = Target_protein - Target_protein_predicted
        
        return [residual_TF, residual_Target]
    
    def calculate_steady_state_outputs(TF_protein_ss, Target_protein_ss):
        """Calculate all steady-state quantities given protein levels"""
        # Set K values for output calculation
        K_TF_val = np.sqrt(max_effect-1)*TF_protein_ss if K_TF == 'auto' else K_TF
        K_Target_val = np.sqrt(max_effect-1)*Target_protein_ss if K_Target == 'auto' else K_Target
        
        # Recalculate Hill functions
        Target_hill = (Target_protein_ss**n_TF) / (K_TF_val**n_TF + Target_protein_ss**n_TF)
        TF_hill = (TF_protein_ss**n_Target) / (K_Target_val**n_Target + TF_protein_ss**n_Target)
        
        # Calculate transcription rates
        TF_transcription_rate = basal_TF_rate 
        Target_transcription_rate = basal_Target_rate
        
        # Calculate effective k_on rates
        k_on_TF_effective = k_on_TF #* max_effect * Target_hill
        k_on_Target_effective = k_on_Target * max_effect * TF_hill
        
        # Calculate all intermediate quantities
        # TF pathway
        TF_bursting_fraction = k_on_TF_effective / (k_on_TF_effective + k_off_TF)
        TF_unspliced_mRNA = (TF_transcription_rate * TF_bursting_fraction / 
                            (TF_mRNA_degradation_rate + splicing_rate_TF))
        TF_spliced_mRNA = (TF_unspliced_mRNA * splicing_rate_TF / 
                          TF_mRNA_degradation_rate)
        
        # Target pathway
        Target_bursting_fraction = k_on_Target_effective / (k_on_Target_effective + k_off_Target)
        Target_unspliced_mRNA = (Target_transcription_rate * Target_bursting_fraction / 
                                (Target_mRNA_degradation_rate + splicing_rate_Target))
        Target_spliced_mRNA = (Target_unspliced_mRNA * splicing_rate_Target / 
                              Target_mRNA_degradation_rate)
        
        return {
            'TF_protein': TF_protein_ss,
            'TF_spliced_mRNA': TF_spliced_mRNA,
            'TF_unspliced_mRNA': TF_unspliced_mRNA,
            'TF_transcription_rate': TF_transcription_rate,
            'TF_bursting_fraction': TF_bursting_fraction,
            'Target_protein': Target_protein_ss,
            'Target_spliced_mRNA': Target_spliced_mRNA,
            'Target_unspliced_mRNA': Target_unspliced_mRNA,
            'Target_transcription_rate': Target_transcription_rate,
            'Target_bursting_fraction': Target_bursting_fraction,
            'Target_hill_function': TF_hill,
            'TF_hill_function': Target_hill,
            'K_TF_used': K_TF_val,
            'K_Target_used': K_Target_val,
            'k_on_TF_effective': k_on_TF_effective,
            'k_on_Target_effective': k_on_Target_effective
        }
    
    # Set up initial guesses
    if initial_guess is None:
        # Default: try multiple initial guesses
        initial_guesses = [
            [10.0, 10.0],           # Both moderate
            [0.1, 0.1],           # Both low
            [100.0, 100.0],         # Both high
            [0.1, 100.0],          # TF low, Target high
            [100.0, 0.1],          # TF high, Target low
        ]
    else:
        initial_guesses = [initial_guess]
    
    solutions = []
    solver_info = []
    
    # Try to find solutions with different initial guesses
    for i, guess in enumerate(initial_guesses):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                
                if method == 'fsolve':
                    sol = fsolve(steady_state_system, guess, full_output=True)
                    solution, info, ier, msg = sol
                    success = (ier == 1)
                else:
                    sol = root(steady_state_system, guess, method=method)
                    solution = sol.x
                    success = sol.success
                    msg = sol.message if hasattr(sol, 'message') else 'No message'
                
                # Check if solution is valid (positive proteins, small residual)
                if success and all(x > 0 for x in solution):
                    residual = steady_state_system(solution)
                    residual_norm = np.linalg.norm(residual)
                    
                    if residual_norm < 1e-6:  # Solution tolerance
                        # Check if this solution is already found
                        is_new = True
                        for existing_sol, _ in solutions:
                            if np.allclose(solution, existing_sol, rtol=1e-4):
                                is_new = False
                                break
                        
                        if is_new:
                            solutions.append((solution, residual_norm))
                            solver_info.append({
                                'initial_guess': guess,
                                'residual_norm': residual_norm,
                                'message': msg,
                                'method': method
                            })
                            
                            if not find_all_solutions:
                                break
        
        except Exception as e:
            continue
    
    if not solutions:
        raise ValueError(f"No steady-state solution found after {len(initial_guesses)} attempts. "
                        "Try adjusting parameters or initial guesses.")
    
    # Return the best solution (lowest residual)
    best_solution, best_residual = min(solutions, key=lambda x: x[1])
    TF_protein_ss, Target_protein_ss = best_solution
    
    # Calculate all steady-state quantities
    results = calculate_steady_state_outputs(TF_protein_ss, Target_protein_ss)
    
    # Add solver information
    results['solver_info'] = {
        'num_solutions_found': len(solutions),
        'best_residual': best_residual,
        'all_solutions': solutions if find_all_solutions else [best_solution],
        'solver_details': solver_info
    }
    
    return results

# Example usage and test function
def example_mutual_activation():
    """Example: Two genes that activate each other with self-consistent K values"""
    
    params = {
        # TF gene parameters
        'k_on_TF': 0.25,
        'k_off_TF': 8.4,
        'basal_TF_rate': 32 * 8.4,
        'TF_mRNA_degradation_rate': np.log(2)/2.5,
        'splicing_rate_TF': np.log(2) / (7 / 60),
        'TF_protein_production_rate': 0.059,
        'TF_protein_degradation_rate': np.log(2) / 28, 
        
        # Target gene parameters
        'k_on_Target': 0.25,
        'k_off_Target': 8.4,
        'basal_Target_rate': 32 * 8.4,
        'Target_mRNA_degradation_rate': np.log(2)/2.5,
        'splicing_rate_Target':  np.log(2) / (7 / 60),
        'Target_protein_production_rate': 0.059,
        'Target_protein_degradation_rate': np.log(2) / 28,
        
        # Hill function parameters - using self-consistent K values
        'K_TF': 'auto',        # K will equal TF steady-state protein level
        'K_Target': 'auto',    # K will equal Target steady-state protein level
        'n_TF': 2,
        'n_Target': 2,
        
        # Solver options
        'find_all_solutions': True
    }
    
    result = get_coupled_steady_state(**params)
    
    print("Steady-state solution with self-consistent K values and effective k_on:")
    print(f"TF protein: {result['TF_protein']:.3f}")
    print(f"Target protein: {result['Target_protein']:.3f}")
    print(f"K_TF used: {result['K_TF_used']:.3f} (= TF protein level)")
    print(f"K_Target used: {result['K_Target_used']:.3f} (= Target protein level)")
    print(f"k_on_TF_effective: {result['k_on_TF_effective']:.4f} (vs base {params['k_on_TF']:.3f})")
    print(f"k_on_Target_effective: {result['k_on_Target_effective']:.4f} (vs base {params['k_on_Target']:.3f})")
    print(f"TF mRNA: {result['TF_spliced_mRNA']:.3f}")
    print(f"Target mRNA: {result['Target_spliced_mRNA']:.3f}")
    print(f"TF Hill function value: {result['TF_hill_function']:.3f}")
    print(f"Target Hill function value: {result['Target_hill_function']:.3f}")
    print(f"Solutions found: {result['solver_info']['num_solutions_found']}")
    print(f"Residual: {result['solver_info']['best_residual']:.2e}")
    
    return result

if __name__ == "__main__":
    example_mutual_activation()

Steady-state solution with self-consistent K values and effective k_on:
TF protein: 63.803
Target protein: 150.041
K_TF used: 247.110 (= TF protein level)
K_Target used: 581.108 (= Target protein level)
k_on_TF_effective: 0.2500 (vs base 0.250)
k_on_Target_effective: 0.0476 (vs base 0.250)
TF mRNA: 26.771
Target mRNA: 5.224
TF Hill function value: 0.269
Target Hill function value: 0.012
Solutions found: 1
Residual: 2.84e-14


In [6]:
import numpy as np
from scipy.integrate import solve_ivp

def get_steady_state_proteins(
    k_on_TF, k_off_TF, TF_transcription_rate, TF_mRNA_degradation_rate, splicing_rate, TF_protein_production_rate, TF_protein_degradation_rate,
    k_on_Target, k_off_Target, Target_transcription_rate, Target_mRNA_degradation_rate, Target_protein_production_rate, Target_protein_degradation_rate,
    max_effect=16.0, TF_hill=0.5, t_max=1000
):
    """Returns TF and Target steady-state protein levels."""
    
    # def odes(t, y):
    #     TF_u, TF_s, TF_p, T_u, T_s, T_p = y
        
    #     # Effective rates
    #     k_TF_eff = k_on_TF
    #     k_T_eff = k_on_Target * max_effect * TF_hill
        
    #     # Burst fractions
    #     b_TF = k_TF_eff / (k_TF_eff + k_off_TF)
    #     b_T = k_T_eff / (k_T_eff + k_off_Target)
        
    #     return [
    #         basal_TF_rate * b_TF - TF_u * (TF_mRNA_deg + splicing_TF),
    #         TF_u * splicing_TF - TF_s * TF_mRNA_deg,
    #         TF_s * TF_prod - TF_p * TF_deg,
    #         basal_Target_rate * b_T - T_u * (Target_mRNA_deg + splicing_Target),
    #         T_u * splicing_Target - T_s * Target_mRNA_deg,
    #         T_s * Target_prod - T_p * Target_deg
    #     ]
    def odes(t, y):
        TF_u, TF_s, TF_p, T_u, T_s, T_p = y
        
        # Effective rates (symmetric regulation)
        k_TF_eff = k_on_TF * max_effect * TF_hill  # Target regulates TF
        k_T_eff = k_on_Target * max_effect * TF_hill  # TF regulates Target
        
        # Burst fractions
        b_TF = k_TF_eff / (k_TF_eff + k_off_TF)
        b_T = k_T_eff / (k_T_eff + k_off_Target)
        
        return [
            TF_transcription_rate * b_TF - TF_u * (TF_mRNA_degradation_rate + splicing_rate),
            TF_u * splicing_rate - TF_s * TF_mRNA_degradation_rate,
            TF_s * TF_protein_production_rate - TF_p * TF_protein_degradation_rate,
            Target_transcription_rate * b_T - T_u * (Target_mRNA_degradation_rate + splicing_rate),
            T_u * splicing_rate - T_s * Target_mRNA_degradation_rate,
            T_s * Target_protein_production_rate - T_p * Target_protein_degradation_rate
        ]
    
    sol = solve_ivp(odes, [0, t_max], [1, 1, 1, 1, 1, 1], method='RK45', rtol=1e-8)
    return sol.y[2, -1], sol.y[5, -1]  # TF_protein, Target_protein

# Example
TF_ss, Target_ss = get_steady_state_proteins(
    0.25, 8.4, 268.8, np.log(2)/2.5, np.log(2)/(7/60), 0.059, np.log(2)/28,
    0.25, 8.4, 268.8, np.log(2)/2.5, 0.059, np.log(2)/28
)
print(f"TF: {TF_ss:.3f}, Target: {Target_ss:.3f}")

TF: 424.538, Target: 424.538


In [3]:
63.803/424.538

0.15028807786346568

In [7]:
# TF gene parameters
param_dict = {
'k_on_TF': 0.25,
'k_off_TF': 8.4,
'basal_TF_rate': 32 * 8.4,
'TF_mRNA_degradation_rate': np.log(2)/2.5,
'splicing_rate_TF': np.log(2) / (7 / 60),
'TF_protein_production_rate': 0.059,
'TF_protein_degradation_rate': np.log(2) / 28, 

# Target gene parameters
'k_on_Target': 0.25,
'k_off_Target': 8.4,
'basal_Target_rate': 32 * 8.4,
'Target_mRNA_degradation_rate': np.log(2)/2.5,
'splicing_rate_Target':  np.log(2) / (7 / 60),
'Target_protein_production_rate': 0.059,
'Target_protein_degradation_rate': np.log(2) / 28,

# Hill function parameters - using self-consistent K values
'K_TF': 'auto',        # K will equal TF steady-state protein level
'K_Target': 'auto',    # K will equal Target steady-state protein level
'n_TF': 2,
'n_Target': 2}

In [8]:
mRNA_level = (8*param_dict['k_on_TF'] /(8*param_dict['k_on_TF'] + param_dict['k_off_TF']))*(param_dict['basal_TF_rate']/(param_dict['TF_mRNA_degradation_rate'] + param_dict['splicing_rate_TF']))
spliced_mRNA_level = mRNA_level * (param_dict['splicing_rate_TF'] / param_dict['TF_mRNA_degradation_rate'])
protein_level = spliced_mRNA_level * (param_dict['TF_protein_production_rate'] / param_dict['TF_protein_degradation_rate'])
print(f"TF mRNA level: {mRNA_level:.3f}, Spliced mRNA level: {spliced_mRNA_level:.3f}, Protein level: {protein_level:.3f}")

TF mRNA level: 8.313, Spliced mRNA level: 178.128, Protein level: 424.538


In [15]:
#to get TF protein mean for TF protein->Target k_on link function
def get_averages(k_on_TF=None,
                 k_off_TF=None,
                 TF_transcription_rate=None,
                 TF_mRNA_degradation_rate=None,
                 splicing_rate=None,
                 protein_production_rate=None,
                 protein_degradation_rate=None,
                 **other_kwargs):
    average_bursting = k_on_TF / (k_on_TF + k_off_TF)
    average_unspliced_mRNA = TF_transcription_rate * average_bursting / (TF_mRNA_degradation_rate + splicing_rate)
    average_spliced_mRNA = average_unspliced_mRNA * splicing_rate / TF_mRNA_degradation_rate
    average_protein = average_spliced_mRNA * protein_production_rate / protein_degradation_rate
    return dict(average_spliced_mRNA=average_spliced_mRNA, average_protein=average_protein)

k_on_TF = 0.25
k_off_TF = 8.4
TF_transcription_rate = 32 * k_off_TF
TF_mRNA_degradation_rate = np.log(2) / 2.5
splicing_rate = np.log(2) / (7 / 60)
protein_production_rate = 0.059
TF_protein_degradation_rate = np.log(2) / 28
a = get_averages(k_on_TF=k_on_TF,
                    k_off_TF=k_off_TF,
                    TF_transcription_rate=TF_transcription_rate,
                    TF_mRNA_degradation_rate=TF_mRNA_degradation_rate,
                    splicing_rate=splicing_rate,
                    protein_production_rate=protein_production_rate
                    , protein_degradation_rate=TF_protein_degradation_rate)
a.items()

dict_items([('average_spliced_mRNA', 26.770674887082127), ('average_protein', 63.80341167619444)])

In [16]:
kOnEff = k_on_TF
bursting_fraction = kOnEff / (kOnEff + k_off_TF)
mRNA_prod = TF_transcription_rate * bursting_fraction / (TF_mRNA_degradation_rate + splicing_rate)
splicesd_mRNA = mRNA_prod * splicing_rate / TF_mRNA_degradation_rate
protein_level = splicesd_mRNA * protein_production_rate / TF_protein_degradation_rate
print(splicesd_mRNA)

26.770674887082127
